## Image corpus creation and Transfer Learning in CNN

##### 1. Import the necessary libraries

In [1]:
import datetime
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import RMSprop,SGD,Adam

##### 2. Initialize some parameters

In [2]:
now = datetime.datetime.now
batch_size = 128
num_classes = 5
epochs = 5

img_rows, img_cols =28,28
filters = 32
pool_size = 2
kernel_size = 3

##### 3. Partition MNIST dataset

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train_1t5 = x_train[y_train < 5]
y_train_1t5 = y_train[y_train < 5]
x_test_1t5 = x_test[y_test < 5]
y_test_1t5 = y_test[y_test < 5]

x_train_gte5 = x_train[y_train >= 5]
y_train_gte5 = y_train[y_train >= 5] - 5
x_test_gte5 = x_test[y_test >= 5]
y_test_gte5 = y_test[y_test >= 5] - 5

In [4]:
print(x_train_1t5.shape)
print(x_test_1t5.shape)

(30596, 28, 28)
(5139, 28, 28)


In [5]:
print(x_train_gte5.shape)
print(x_test_gte5.shape)

(29404, 28, 28)
(4861, 28, 28)


##### 4. Define the feature layers

In [6]:
Input_Shape = (28,28,1)
model = Sequential()
feature_layers=[Conv2D(filters=filters,kernel_size=kernel_size,activation='relu',padding='same',input_shape=Input_Shape),
                Conv2D(filters=filters,kernel_size=kernel_size,activation='relu',input_shape=Input_Shape),
                MaxPooling2D(pool_size=pool_size),
                Dropout(0.5),
                Flatten()]

##### 5. Define the classification layers

In [7]:
classification_layers = [Dense(128,activation='relu'),
                         Dropout(0.5),
                         Dense(num_classes,activation='softmax')]

##### 6. Define a sequential model

In [8]:
model1 = Sequential(feature_layers + classification_layers)
model1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 26, 26, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 13, 13, 32)        0         
                                                                 
 flatten (Flatten)           (None, 5408)              0         
                                                                 
 dense (Dense)               (None, 128)               692352    
                                                      

##### 7. Create a function train model

In [9]:
def train_model1(model1,train,test,num_classes):
    train[0] = train[0].reshape(train[0].shape[0],img_rows,img_cols,1)
    test[0] = test[0].reshape(test[0].shape[0],img_rows,img_cols,1)
    train[0] = train[0].astype('float32')
    test[0] = test[0].astype('float32')

    train[0] /= 255
    test[0] /= 255
    print(train[0].shape)
    print(test[0].shape)
    s = pd.Series(train[1])
    train[1] = pd.get_dummies(s)
    train[1] = train[1].values
    s = pd.Series(test[1])
    test[1] = pd.get_dummies(s)
    test[1] = test[1].values
    
    model1.compile(optimizer='adadelta',loss='categorical_crossentropy',metrics=['accuracy'])
    train_start_time=now()
    model1.fit(train[0],train[1],epochs=epochs,verbose=2)

    print('\n')
    print('--------------------------------------------------------------------------------')
    print('\n')

    print('Training time: %s' % (now() - train_start_time))

    score=model1.evaluate(test[0],test[1],verbose=0)

    print('test loss ',score[0])
    print('test accuracy ',score[1])

##### 8. Training on digits 5 to 9

In [10]:
import numpy as np
import pandas as pd

In [11]:
train_model1(model1,[x_train_gte5,y_train_gte5],[x_test_gte5,y_test_gte5],num_classes)

(29404, 28, 28, 1)
(4861, 28, 28, 1)
Epoch 1/5
919/919 - 29s - loss: 1.5806 - accuracy: 0.2769 - 29s/epoch - 31ms/step
Epoch 2/5
919/919 - 27s - loss: 1.5374 - accuracy: 0.3677 - 27s/epoch - 30ms/step
Epoch 3/5
919/919 - 27s - loss: 1.4861 - accuracy: 0.4526 - 27s/epoch - 30ms/step
Epoch 4/5
919/919 - 27s - loss: 1.4194 - accuracy: 0.5185 - 27s/epoch - 30ms/step
Epoch 5/5
919/919 - 27s - loss: 1.3372 - accuracy: 0.5783 - 27s/epoch - 30ms/step


--------------------------------------------------------------------------------


Training time: 0:02:18.402042
test loss  1.226709008216858
test accuracy  0.7634231448173523


##### 9. Freeze Feature Layers

In [12]:
for l in feature_layers:
    l.trainable = False

##### 10. Print Summary

In [13]:
model1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 26, 26, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 13, 13, 32)        0         
                                                                 
 flatten (Flatten)           (None, 5408)              0         
                                                                 
 dense (Dense)               (None, 128)               692352    
                                                      

##### 11. Training for digits 0 to 4 based on digits 5 to 9

In [14]:
train_model1(model1,[x_train_1t5, y_train_1t5],[x_test_1t5, y_test_1t5], num_classes)

(30596, 28, 28, 1)
(5139, 28, 28, 1)
Epoch 1/5
957/957 - 12s - loss: 1.5314 - accuracy: 0.3444 - 12s/epoch - 13ms/step
Epoch 2/5
957/957 - 12s - loss: 1.4198 - accuracy: 0.4648 - 12s/epoch - 12ms/step
Epoch 3/5
957/957 - 12s - loss: 1.3239 - accuracy: 0.5659 - 12s/epoch - 12ms/step
Epoch 4/5
957/957 - 12s - loss: 1.2285 - accuracy: 0.6551 - 12s/epoch - 12ms/step
Epoch 5/5
957/957 - 12s - loss: 1.1392 - accuracy: 0.7257 - 12s/epoch - 13ms/step


--------------------------------------------------------------------------------


Training time: 0:00:59.628315
test loss  1.017417550086975
test accuracy  0.8764351010322571


##### 12. Reversing the training process

In [15]:
INPUT_SHAPE = (img_rows,img_cols,1)
feature_layers=[Conv2D(filters=filters,kernel_size=kernel_size,activation='relu',input_shape=Input_Shape),
                 Conv2D(filters=filters,kernel_size=kernel_size,activation='relu',input_shape=Input_Shape),
                 MaxPooling2D(pool_size=pool_size),
                 Dropout(0.25),
                 Flatten()]

classification_layers = [Dense(128,activation='relu'),Dropout(0.5),Dense(5,activation='softmax')] 

In [16]:
modelReverse = Sequential(feature_layers+classification_layers)
modelReverse.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 conv2d_3 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 12, 12, 32)       0         
 2D)                                                             
                                                                 
 dropout_2 (Dropout)         (None, 12, 12, 32)        0         
                                                                 
 flatten_1 (Flatten)         (None, 4608)              0         
                                                                 
 dense_2 (Dense)             (None, 128)               589952    
                                                      

In [17]:
train_model1(modelReverse,[x_train_1t5,y_train_1t5],[x_test_1t5,y_test_1t5],num_classes)

(30596, 28, 28, 1)
(5139, 28, 28, 1)
Epoch 1/5
957/957 - 27s - loss: 1.5787 - accuracy: 0.2934 - 27s/epoch - 28ms/step
Epoch 2/5
957/957 - 26s - loss: 1.4798 - accuracy: 0.5289 - 26s/epoch - 27ms/step
Epoch 3/5
957/957 - 25s - loss: 1.3421 - accuracy: 0.6753 - 25s/epoch - 27ms/step
Epoch 4/5
957/957 - 25s - loss: 1.1548 - accuracy: 0.7574 - 25s/epoch - 26ms/step
Epoch 5/5
957/957 - 26s - loss: 0.9398 - accuracy: 0.8130 - 26s/epoch - 27ms/step


--------------------------------------------------------------------------------


Training time: 0:02:09.627054
test loss  0.744502604007721
test accuracy  0.9264448285102844


In [18]:
for l in feature_layers:
    l.trainable = False

In [19]:
modelReverse.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 conv2d_3 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 12, 12, 32)       0         
 2D)                                                             
                                                                 
 dropout_2 (Dropout)         (None, 12, 12, 32)        0         
                                                                 
 flatten_1 (Flatten)         (None, 4608)              0         
                                                                 
 dense_2 (Dense)             (None, 128)               589952    
                                                      

In [20]:
train_model1(modelReverse,[x_train_gte5,y_train_gte5],[x_test_gte5,y_test_gte5],num_classes)

(29404, 28, 28, 1)
(4861, 28, 28, 1)
Epoch 1/5
919/919 - 10s - loss: 1.6021 - accuracy: 0.3159 - 10s/epoch - 11ms/step
Epoch 2/5
919/919 - 10s - loss: 1.4153 - accuracy: 0.4123 - 10s/epoch - 11ms/step
Epoch 3/5
919/919 - 10s - loss: 1.2621 - accuracy: 0.5242 - 10s/epoch - 11ms/step
Epoch 4/5
919/919 - 10s - loss: 1.1390 - accuracy: 0.6151 - 10s/epoch - 11ms/step
Epoch 5/5
919/919 - 10s - loss: 1.0361 - accuracy: 0.6776 - 10s/epoch - 11ms/step


--------------------------------------------------------------------------------


Training time: 0:00:52.240193
test loss  0.9111851453781128
test accuracy  0.7881094217300415


## CASE 1 : Digits 5-9 is trained then last layer of network trained on Digit 0-4 

### Digit 5-9 : 

Training time: 0:01:45.134918

test loss  1.2558680772781372

test accuracy  0.7595145106315613



### Digit 0-4 : 

Training time: 0:00:24.913844

test loss  0.9772664904594421

test accuracy  0.8803269267082214



## CASE 2 : Digits 0-4 is trained then last layer of network trained on Digit 5-9

### Digit 0-4 :

Training time: 0:00:21.262768

test loss  0.7906395196914673

test accuracy  0.8111499547958374


### Digit 5-9: 

Training time: 0:01:34.288307

test loss  0.601348876953125

test accuracy  0.935785174369812
